In [33]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import torch
import skorch
import scipy
import torch.nn as nn
import torch
import torch.nn.functional as F
import sys
from skorch.helper import DataFrameTransformer
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn import metrics
from sklearn.preprocessing import FunctionTransformer

# Please ignore the below cells, they were for additional preprocessing and do not need to be run again

In [54]:
# Opening JSON file
f = open('twitter_aapl_1.json',)
  
# returns JSON object as 
# a dictionary
aapl = json.load(f)

f.close()

In [55]:
flataapl  = [val for sublist in aapl for val in sublist]

In [56]:
print(flataapl[0])

{'created_at': '2021-05-11T10:18:35.000Z', 'id': '1392061565185236994', 'text': 'RT @Nicochan33: Apple Execs Chose to Keep a Hack of 128 Million iPhones Quiet https://t.co/waR9tgHKCA #tech #feedly #apple #iphone #cyberse…'}


In [57]:
df = pd.DataFrame(flataapl)
df

,created_at,id,text,withheld
0,2021-05-11T10:18:35.000Z,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...,NaN
1,2021-05-11T10:18:30.000Z,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr...",NaN
2,2021-05-11T10:18:25.000Z,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...,NaN
3,2021-05-11T10:17:54.000Z,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...,NaN
4,2021-05-11T10:17:48.000Z,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...,NaN
...,...,...,...,...
517126,2021-02-18T07:25:14.000Z,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...,NaN
517127,2021-02-18T07:25:08.000Z,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...,NaN
517128,2021-02-18T07:25:06.000Z,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...,NaN
517129,2021-02-18T07:24:56.000Z,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd,NaN


In [58]:
y = df['withheld'].dropna()
y

152369    {'copyright': False, 'country_codes': ['IN']}
257375    {'copyright': False, 'country_codes': ['IN']}
269997    {'copyright': False, 'country_codes': ['IN']}
315545    {'copyright': False, 'country_codes': ['RU']}
315578    {'copyright': False, 'country_codes': ['RU']}
343775    {'copyright': False, 'country_codes': ['IN']}
473481    {'copyright': False, 'country_codes': ['RU']}
473719    {'copyright': False, 'country_codes': ['RU']}
489545    {'copyright': False, 'country_codes': ['IN']}
509238    {'copyright': False, 'country_codes': ['IN']}
Name: withheld, dtype: object

In [59]:
df1 = df.drop('withheld',axis=1)
df1['created_at'] = df1['created_at'].map(lambda x: str(x)[:-5])
df1

,created_at,id,text
0,2021-05-11T10:18:35,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...
1,2021-05-11T10:18:30,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr..."
2,2021-05-11T10:18:25,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...
3,2021-05-11T10:17:54,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...
4,2021-05-11T10:17:48,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...
...,...,...,...
517126,2021-02-18T07:25:14,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...
517127,2021-02-18T07:25:08,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...
517128,2021-02-18T07:25:06,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...
517129,2021-02-18T07:24:56,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd


In [60]:
df1.to_csv("twitter_aapl_1_clean.csv", index = False)

In [2]:
df1 = pd.read_csv("twitter_aapl_1_clean.csv")
# Only including this step for now, basically removes the time component, leaving only date
df1['created_at'] = df1['created_at'].map(lambda x: str(x)[:-9])
df1 = df1.rename(columns={"created_at" : "Date"})
df1

,Date,id,text
0,2021-05-11,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...
1,2021-05-11,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr..."
2,2021-05-11,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...
3,2021-05-11,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...
4,2021-05-11,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...
...,...,...,...
517126,2021-02-18,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...
517127,2021-02-18,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...
517128,2021-02-18,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...
517129,2021-02-18,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd


In [3]:
# adapted from https://www.30secondsofcode.org/python/s/is-weekend
def if_weekend(d = datetime.today()):
    if d.weekday() == 5:
        da = d + timedelta(days=2)
        return da
    elif d.weekday() == 6:
        da = d + timedelta(days=1)
        return da
    elif d == date(2021,4,2):
        return date(2021,4,5)
    else:
        return d
print(if_weekend(date(2021,6,14)))
print(type(date(2021,6,14)))

2021-06-14
<class 'datetime.date'>


In [4]:
df1['Date']= pd.to_datetime(df1['Date'])
df1['usedate'] = df1['Date'].map(lambda x: if_weekend(x))
df1

,Date,id,text,usedate
0,2021-05-11,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...,2021-05-11
1,2021-05-11,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr...",2021-05-11
2,2021-05-11,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...,2021-05-11
3,2021-05-11,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...,2021-05-11
4,2021-05-11,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...,2021-05-11
...,...,...,...,...
517126,2021-02-18,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...,2021-02-18
517127,2021-02-18,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...,2021-02-18
517128,2021-02-18,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...,2021-02-18
517129,2021-02-18,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd,2021-02-18


In [5]:
price = pd.read_csv("aapl2.csv")
price['Date'] = price['Date'].map(lambda x: parser.parse(x).isoformat())
# Only including this step for now, basically removes the time component, leaving only date
price['Date'] = price['Date'].map(lambda x: str(x)[:-9])
price['Date']= pd.to_datetime(price['Date'])
# Change me if looking to change examination scope
price = price.rename(columns={"Date" : "usedate"})
price

,usedate,Close/Last,Volume,Open,High,Low,Change
0,2021-05-11,125.9100,126142800,123.5000,126.2700,122.7700,2.4100
1,2021-05-10,126.8500,88071230,129.4100,129.5400,126.8100,-2.5600
2,2021-05-07,130.2100,78973270,130.8500,131.2582,129.4750,-0.6400
3,2021-05-06,129.7400,78128330,127.8900,129.7500,127.1300,1.8500
4,2021-05-05,128.1000,84000900,129.2000,130.4500,127.9700,-1.1000
...,...,...,...,...,...,...,...
247,2020-05-18,78.7400,135372520,78.2925,79.1250,77.5810,0.4475
248,2020-05-15,76.9275,166348360,75.0875,76.9750,75.0525,1.8400
249,2020-05-14,77.3850,158929080,76.1275,77.4475,75.3825,1.2575
250,2020-05-13,76.9125,200622560,78.0375,78.9875,75.8025,-1.1250


In [6]:
merged = pd.merge(df1, price, how='left', on='usedate')
merged

,Date,id,text,usedate,Close/Last,Volume,Open,High,Low,Change
0,2021-05-11,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
1,2021-05-11,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr...",2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
2,2021-05-11,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
3,2021-05-11,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
4,2021-05-11,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
...,...,...,...,...,...,...,...,...,...,...
517126,2021-02-18,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51
517127,2021-02-18,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51
517128,2021-02-18,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51
517129,2021-02-18,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51


In [7]:
merged.isna().sum()

Date          0
id            0
text          0
usedate       0
Close/Last    0
Volume        0
Open          0
High          0
Low           0
Change        0
dtype: int64

In [24]:
# merged.to_csv("merged.csv", index = False)

# Start Here

In [3]:
merged = pd.read_csv('merged.csv')
# Obtaining the tweet contents into a list
all_tweets = merged["text"]
print(all_tweets)
all_tweets = all_tweets.to_list()
print(all_tweets[0])
print(all_tweets[1])
print(all_tweets[2])

0         RT @Nicochan33: Apple Execs Chose to Keep a Ha...
1         RT @RoseLoveStyle: House of the Dragon, the pr...
2         RT @gtorges: Ich habe jetzt einiges an Materia...
3         RT @iTech911: Future versions of #Apple's CarK...
4         発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...
                                ...                        
517126    RT @mobitrade_: Original Apple Airpods Pro New...
517127    #ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...
517128    Sommige appels worden te snel slecht.\n\nhttps...
517129    #apple #kia #StockMarkets https://t.co/ccRBBAKDtd
517130    RT @mobitrade_: iPhone 8 Neatly Used 64GB Fact...
Name: text, Length: 517131, dtype: object
RT @Nicochan33: Apple Execs Chose to Keep a Hack of 128 Million iPhones Quiet https://t.co/waR9tgHKCA #tech #feedly #apple #iphone #cyberse…
RT @RoseLoveStyle: House of the Dragon, the prequel to Game of Thrones, was announced by Casey Bloys, president of HBO. #gamesofthrones 
 #…
RT @gtorges: Ich

In [8]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nathan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [4]:
# https://python.gotrained.com/scraping-tweets-sentiment-analysis/
# Basic cleaning of text before TF-IDF; this process will be improved later
# for tweet in all_tweets:
#     # Remove all the special characters
#     processed_tweet = re.sub(r'\W', ' ', tweet)
 
#     # remove all single characters
#     processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
#     # Remove single characters from the start
#     processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
#     # Substituting multiple spaces with single space
#     processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
#     # Removing prefixed 'b'
#     processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
    
#     #removing common pronouns and prepositions
#     processed_tweet = re.sub(r'of|to|https|keep|128', '', processed_tweet)
#     processed_tweet = lemmatizer.lemmatize(processed_tweet)
# #     lambda x: lemmatizer.lemmatize(x in processed_tweet)
#     # Converting to Lowercase
#     processed_tweet = processed_tweet.lower()
    
#     tweet = processed_tweet
    
    
processed_tweets = []
X = all_tweets
for tweet in range(0, len(X)):  
    

    # Remove all the special characters
    processed_tweet = re.sub(r'\W', ' ', str(X[tweet]))
 
    processed_tweet = re.sub(r'http\S+', '', processed_tweet) 
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
    # Remove single characters from the start
    processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
    # Substituting multiple spaces with single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
    # Removing prefixed 'b'
    processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
    

    
    processed_tweet = re.sub(r'of|to|https|keep', '', processed_tweet)
    
#     processed_tweet = lemmatizer.lemmatize(processed_tweet)
#     lambda x: lemmatizer.lemmatize(x in processed_tweet)
    # Converting to Lowercase
    processed_tweet = processed_tweet.lower()
 
    processed_tweets.append(processed_tweet)

In [5]:
print(processed_tweets[0])

rt nicochan33 apple execs chose  keep hack  128 million iphones quiet co war9tghkca tech feedly apple iphone cyberse 


In [8]:

def identity_tokenizer(text):
    return text
tfidfv = TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')
df2 = tfidfv.fit_transform(processed_tweets)
print(df2)
# df2array = df2.toarray()
# print(df2array)

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  (0, 12)	0.10001855289941801
  (0, 34)	0.1777075423883674
  (0, 14)	0.08544893856499532
  (0, 16)	0.10444099928001752
  (0, 17)	0.09326166874737839
  (0, 10)	0.19628660456754524
  (0, 32)	0.10439659684530907
  (0, 30)	0.09123751372315966
  (0, 26)	0.1639570527498012
  (0, 22)	0.08557299792767131
  (0, 9)	0.2018444861394874
  (0, 3)	0.15470239282849682
  (0, 2)	0.13460605986123836
  (0, 20)	0.22903525699541016
  (0, 28)	0.19391006851380474
  (0, 33)	0.1463870089382474
  (0, 15)	0.28333186750616846
  (0, 21)	0.20147651250489293
  (0, 25)	0.22918013166103215
  (0, 4)	0.30706321045206975
  (0, 18)	0.2569280542171668
  (0, 24)	0.21639533847862374
  (0, 13)	0.253285464329252
  (0, 23)	0.20936593149346489
  (0, 0)	0.3041733475265926
  :	:
  (517130, 11)	0.1597365041123458
  (517130, 12)	0.14757456084735618
  (517130, 34)	0.12494088618743826
  (517130, 14)	0.14336046748648082
  (517130, 16)	0.07342935943500598
  (517130, 17)	0.11101882205884296
  (517130, 10)	0.3601109701835194
  (517130, 30)

In [21]:
print(type(df2))
print(type(df2[-1]))
print(type(df2[-1][-1]))
print(df2[-1][-1])

<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
  (0, 556)	0.40357727068148264
  (0, 88)	0.34308710016633726
  (0, 97)	0.417333363196858
  (0, 443)	0.3951839877481259
  (0, 674)	0.3663719517401211
  (0, 1770)	0.3199229308458355
  (0, 1783)	0.31050472625527564
  (0, 943)	0.15704072326243243
  (0, 1716)	0.15129041849415864
  (0, 1471)	0.09239185493528781


In [11]:
print(df2.size)

6813202


In [12]:
print(df2[0])

print(df2array[0])
print(type(df2[0][0]))

  (0, 943)	0.1619164850216469
  (0, 1667)	0.24642359750991072
  (0, 412)	0.07984327940748012
  (0, 863)	0.07926270652557654
  (0, 957)	0.3644418579168204
  (0, 1148)	0.37252279088881424
  (0, 24)	0.44364094790152897
  (0, 1243)	0.19111331976332038
  (0, 802)	0.4405447542842643
  (0, 1005)	0.38259748006594846
  (0, 1716)	0.15598764620493955
  (0, 189)	0.14697293203863546
  (0, 1471)	0.09526041452797107
[0. 0. 0. ... 0. 0. 0.]
<class 'scipy.sparse.csr.csr_matrix'>


In [31]:
print(type(df2array))
np.set_printoptions(threshold=sys.maxsize)
print(df2array[0])
np.set_printoptions(threshold = False)

<class 'numpy.ndarray'>
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.44364095 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.       

In [13]:
print(processed_tweets[0:10])

['RT @Nicochan33: Apple Execs Chose to Keep a Hack of 128 Million iPhones Quiet https://t.co/waR9tgHKCA #tech #feedly #apple #iphone #cyberse…', 'RT @RoseLoveStyle: House of the Dragon, the prequel to Game of Thrones, was announced by Casey Bloys, president of HBO. #gamesofthrones \n #…', 'RT @gtorges: Ich habe jetzt einiges an Material zu #Fuellmich &amp; Co. beim @AppleSupport eingereicht.\nHoffentlich verschwindet der #Podcast d…', "RT @iTech911: Future versions of #Apple's CarKey could detect when it's being used near wireless chargers, and alter how it works to avoid…", '発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #忘れん坊 #実は追加でもう一つ買った https://t.co/HSFAmcNdnN', 'RT @Nicochan33: Apple Execs Chose to Keep a Hack of 128 Million iPhones Quiet https://t.co/waR9tgHKCA #tech #feedly #apple #iphone #cyberse…', 'RT @FoxConcours_: 🎁 #Apple #MacBook\n👉 Tente de gagner un #MacBookPro13 en participant a ce concours !\n\n🔸 Pour tenter votre chance :\nRT + Fo…', 'RT @taisy0: Apple、｢Apple TV｣でSiri

In [14]:
print(df2array.sum())
print(type(processed_tweets[0]))

1649856.2906732175
<class 'str'>


In [15]:
print(type(df2))

<class 'scipy.sparse.csr.csr_matrix'>


In [10]:
# https://programmerbackpack.com/tf-idf-explained-and-python-implementation/
dfnew = pd.DataFrame(df2[0].T.todense(), index=tfidfv.get_feature_names(), columns=["TF-IDF"])
dfnew = dfnew.sort_values('TF-IDF', ascending=False)
print (dfnew.head(30))

                 TF-IDF
128            0.443641
hack           0.440545
keep           0.382597
million        0.372523
iphones        0.364442
tech           0.246424
of             0.191113
iphone         0.161916
to             0.155988
apple          0.146973
rt             0.095260
co             0.079843
https          0.079263
photography    0.000000
play           0.000000
platforms      0.000000
playlist       0.000000
players        0.000000
playing        0.000000
playstation    0.000000
playstore      0.000000
platforming    0.000000
platform       0.000000
platformer     0.000000
photooftheday  0.000000
plans          0.000000
pls            0.000000
plan           0.000000
place          0.000000
più            0.000000


In [39]:
# Trying an alternative data implementation, I haven't used this in the later stages yet
df3 = pd.DataFrame.sparse.from_spmatrix(df2)
print(type(df3))
df3

<class 'pandas.core.frame.DataFrame'>


,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
517127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
517128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
517129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
#Selecting the data and splitting into train and test
y = merged['Change']
print(type(y))
# X = df3
# X = df2
# X = df2array
X = processed_tweets
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# train_holdout_data.to_csv('train_holdout_data.csv', index=False)
# x_holdout.to_csv('x_holdout.csv', index=False)
# y_holdout.to_csv('y_holdout.csv', index=False)
# train_data.to_csv('train_data.csv', index=False)
# # x_train.to_csv('x_train.csv', index=False)
# y_train.to_csv('y_train.csv', index=False)
# # x_test.to_csv('x_test.csv', index=False)
# y_test.to_csv('y_test.csv', index=False)

# # train_holdout_data = pd.read_csv('train_holdout_data.csv')
# # x_holdout = pd.read_csv('x_holdout.csv')
# # y_holdout = pd.read_csv('y_holdout.csv')
# # x_train = pd.read_csv('x_train.csv')
# y_train = pd.read_csv('y_train.csv')
# # x_test = pd.read_csv('x_test.csv')
# y_test = pd.read_csv('y_test.csv')


<class 'pandas.core.series.Series'>


In [32]:
# y_train = pd.Series(y_train)
# y_test = pd.Series(y_test)
print(y_test)
print(type(y_train.values))
# print(np.isnan(X).sum())
# print(y.isna().sum())

209358   -1.28
56222     0.50
286159   -0.26
443996    4.04
72293    -0.32
          ... 
477154    2.10
29302     1.85
31067     1.85
1816      2.41
295219    1.05
Name: Change, Length: 103427, dtype: float64
<class 'numpy.ndarray'>


In [14]:
# x_train = torch.from_numpy(x_train)
# x_test = torch.from_numpy(x_test)
# y_train = torch.from_numpy(y_train.values)
# y_test = torch.from_numpy(y_test.values)
print(len(x_train))
# print(x_train)
# print(x_test)
# print(y_train)
# print(y_test)

413704


In [14]:
print(type(x_train))
print(type(x_test))
print(type(y_train))
print(type(y_test))
print(x_train.dtype)
print(x_test.dtype)
print(y_train.dtype)
print(y_test.dtype)
# print(x_test[100000][0])
# if 'X' in x_train:
#     print("hi")

<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
float64
float64


AttributeError: 'DataFrame' object has no attribute 'dtype'

In [23]:
# y_train = y_train.astype(np.float32)
# y_test = y_test.astype(np.float32)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
# y_train = y_train.squeeze()
# y_test = y_test.squeeze()

In [22]:
print(y_train.ndim)

1


In [43]:
# x_train1 = x_train.astype(np.float32)
# y_train1 = y_train.astype(np.float32)
print(x_train.dtype)
print(x_test.dtype)
print(y_train.dtype)
print(y_test.dtype)
print(x_train1.dtype)
print(y_train1.dtype)

AttributeError: 'list' object has no attribute 'dtypes'

In [45]:
# for x in y_train[0]:
#     if type(x) != "class 'numpy.float32'":
#         print("double")
print(type(x_train[0]))
print(type(y_train))
print(type(y_train[0]))
print(type(y_train[0][0]))

<class 'str'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float32'>


AttributeError: 'numpy.ndarray' object has no attribute 'float'

In [20]:
y_trainarray = y_train.values.tolist()
y_testarray = y_test.values.tolist()
# y_trainarray = y_train.to_numpy()
# y_testarray = y_test.to_numpy()

In [17]:
def identity_tokenizer(text):
    return text

tfidfv = TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')
x_train1 = tfidfv.fit_transform(x_train)
x_test1 = tfidfv.fit_transform(x_test)
x_train2 = x_train1.astype(dtype = np.float32)
x_test2 = x_test1.astype(dtype = np.float32)

In [25]:
print(x_train2.shape)
print(y_train.shape)
print(y_train)

(413704, 5205)
(413704, 1)
[[ 1.33]
 [-2.01]
 [ 1.14]
 ...
 [-3.29]
 [-2.73]
 [-1.33]]


In [34]:
# import torch.nn.functional as F


in_dimension = 413704
hid_dimension = 3
out_dimension = 1


class PoleNN(nn.Module):
    def __init__(self):
        super(PoleNN, self).__init__()
        self.fc1 = nn.Linear(in_dimension,hid_dimension)
        self.fc2 = nn.Linear(hid_dimension,out_dimension)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, X):
        hidden = self.fc1(X)
        hidden = self.sigmoid(hidden)
        output = self.fc2(hidden)
        return output

from skorch import NeuralNetRegressor
from sklearn.pipeline import Pipeline

class RegressorModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(5205, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

pole_model = RegressorModule()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(pole_model.parameters(), lr = 0.1)

net = NeuralNetRegressor(module=pole_model, max_epochs=10, lr=0.1)
def typechange(x):
    return x.astype(dtype = np.float32)
typetransform = FunctionTransformer(typechange)
# pipe = Pipeline([('transform', DataFrameTransformer()),('net', net)])
# pipe = Pipeline([('net', net)])
pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("net", net)])


# net = skorch.NeuralNetClassifier(module=PoleNN, max_epochs=20, lr=0.1, criterion=torch.nn.NLLLoss)
pipe.fit(X=x_train, y=y_train)
y_pred = net.predict(x_test2)

  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        4.0862        4.0736  68.9864
      2        4.0719        4.0622  68.7781
      3        4.0573        4.0402  68.8335
      4        4.0306        4.0207  68.2707
      5        4.0161        4.0125  67.9545
      6        4.0060        4.0049  68.2192
      7        3.9970        3.9996  68.2691
      8        3.9903        3.9941  70.2062
      9        3.9837        3.9901  69.6767
     10        3.9767        3.9871  67.8036


RuntimeError: size mismatch, m1: [128 x 4049], m2: [5205 x 10] at C:\Users\builder\AppData\Local\Temp\pip-req-build-e5c8dddg\aten\src\TH/generic/THTensorMath.cpp:136

In [48]:
from sklearn.datasets import make_regression

X_regr, y_regr = make_regression(1000, 20, n_informative=10, random_state=0)
X_regr = X_regr.astype(np.float32)
y_regr = y_regr.astype(np.float32) / 100
y_regr = y_regr.reshape(-1, 1)

X_regr.shape, y_regr.shape, y_regr.min(), y_regr.max()

((1000, 20), (1000, 1), -6.4901485, 6.154505)

In [52]:
print(type(y_regr))
print(type(y_regr[0]))
print(type(y_regr[0][0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float32'>


In [75]:
net.fit(X=X_regr, y=y_regr)

Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        4.6416        3.8738  1.3260
      2        4.3003        2.7721  0.0300
      3        1.7503        0.4882  0.0240
      4        0.4048        0.1588  0.0240
      5        0.1568        0.2447  0.0240
      6        0.2712        0.1374  0.0260
      7        0.1773        0.4521  0.0220
      8        0.2750        0.0977  0.0250
      9        0.0932        0.1167  0.0230
     10        0.0787        0.0926  0.0280


<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=RegressorModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=1, bias=True)
  ),
)

In [ ]:

from sklearn.ensemble import RandomForestRegressor
text_classifier = RandomForestRegressor(n_estimators=100, random_state=0)  
text_classifier.fit(x_train, y_train)


predictions = text_classifier.predict(x_test)
 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
 
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

In [73]:
cross_val_score.get_params()

AttributeError: 'function' object has no attribute 'get_params'

In [ ]:
from sklearn.svm import LinearSVR, SVR

x_trainsvm = x_train
y_trainsvm = y_train
x_testsvm = x_test

y_trainsvm = np.array(y_train).ravel()
y_testsvm = np.array(y_test).ravel()
#Create the SVM model

start = time.time()
# regressor = LinearSVR(random_state = 0)
regressor = SVR()
#Fit the model for the data

regressor.fit(x_trainsvm, y_trainsvm)

#Make the prediction
y_predsvm = regressor.predict(x_testsvm)






accuracies = cross_val_score(estimator = regressor, X = x_trainsvm, y = y_trainsvm, cv = 10)
end = time.time()
# print("Fraction: ", x)
# print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
# print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))
mse = mean_squared_error(y_test,y_predsvm)
print("Mean Squared Error: {}".format(mse))
r2 = r2_score(y_test,y_predsvm)
print("R-squared: {}".format(r2))
print("")
print("Elapsed Time: ", (end-start),"\n")

In [31]:
mse = mean_squared_error(y_test,y_predsvm)
print("Mean Squared Error: {}".format(mse))
r2 = r2_score(y_test,y_predsvm)
print("R-squared: {}".format(r2))

4.00098072231255
Mean Squared Error: 4.00098072231255
R-squared: 0.025518011281076225
